## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
import os


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,53.47,95,100,4.38,overcast clouds
1,1,Tilichiki,RU,60.4667,166.1000,-15.32,88,12,8.86,few clouds
2,2,Porto Novo,BJ,6.4965,2.6036,78.87,84,19,6.87,few clouds
3,3,Jamestown,US,42.0970,-79.2353,21.29,79,100,10.36,overcast clouds
4,4,Bonfim,BR,3.0833,-59.9500,82.67,58,88,9.28,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,53.47,95,100,4.38,overcast clouds
2,2,Porto Novo,BJ,6.4965,2.6036,78.87,84,19,6.87,few clouds
7,7,Busselton,AU,-33.6500,115.3333,64.04,60,0,12.73,clear sky
8,8,Beyneu,KZ,45.3167,55.2000,38.66,83,98,8.68,overcast clouds
14,14,Rafai,CF,4.9500,23.9167,70.84,37,78,2.15,broken clouds
...,...,...,...,...,...,...,...,...,...,...
676,676,Qianjiang,CN,30.4151,112.8859,38.89,88,100,0.94,overcast clouds
681,681,Sumbe,AO,-11.2061,13.8437,75.78,78,85,7.09,overcast clouds
683,683,Sampit,ID,-2.5333,112.9500,73.74,93,100,2.04,overcast clouds
684,684,Kapit,MY,2.0167,112.9333,72.19,100,100,0.98,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                435
City                   435
Country                435
Lat                    435
Lng                    435
Max Temp               435
Humidity               435
Cloudiness             435
Wind Speed             435
Current Description    435
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,53.47,overcast clouds,-46.1927,168.8643,
2,Porto Novo,BJ,78.87,few clouds,6.4965,2.6036,
7,Busselton,AU,64.04,clear sky,-33.6500,115.3333,
8,Beyneu,KZ,38.66,overcast clouds,45.3167,55.2000,
14,Rafai,CF,70.84,broken clouds,4.9500,23.9167,
15,Punta Arenas,CL,53.11,broken clouds,-53.1500,-70.9167,
17,Mar Del Plata,AR,63.09,few clouds,-38.0023,-57.5575,
18,East London,ZA,75.34,clear sky,-33.0153,27.9116,
19,Raha,IN,54.70,clear sky,26.2333,92.5167,
20,Constitucion,CL,60.82,clear sky,-35.3333,-72.4167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
    
    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url=base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,53.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
2,Porto Novo,BJ,78.87,few clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
7,Busselton,AU,64.04,clear sky,-33.6500,115.3333,Observatory Guest House
8,Beyneu,KZ,38.66,overcast clouds,45.3167,55.2000,NUR
14,Rafai,CF,70.84,broken clouds,4.9500,23.9167,
...,...,...,...,...,...,...,...
676,Qianjiang,CN,38.89,overcast clouds,30.4151,112.8859,Shell Qianjiang City Qianjiang bus station Hotel
681,Sumbe,AO,75.78,overcast clouds,-11.2061,13.8437,Hotel Ritz Sumbe
683,Sampit,ID,73.74,overcast clouds,-2.5333,112.9500,Aquarius Boutique Hotel Sampit
684,Kapit,MY,72.19,overcast clouds,2.0167,112.9333,Hotel Meligai Kapit


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))